In [ ]:
%pip install vaderSentiment
%pip install datasets

In [ ]:
from datasets import load_dataset

# Load goEmotions Dataset and get Sentiment Labels
goEmotionsDataset = load_dataset('go_emotions')
sentimentLabels = goEmotionsDataset['train'].features['labels'].feature.names

In [ ]:
sentimentMap = {
  "positive": ["admiration", "amusement", "approval", "caring", "desire", "excitement", "gratitude", "joy", "love", "optimism", "pride", "relief"],
  "negative": ["anger", "annoyance", "disappointment", "disapproval", "disgust", "embarrassment", "fear", "grief", "nervousness", "remorse", "sadness"],
  "neutral": ["neutral", "realization", "surprise", "curiosity", "confusion"]
}

In [ ]:
# Update goEmotions dataset with bucket sentiment categories

def map_sentiment(row):
    bucket = ""

    sentiment_num = row['labels'][0]
    sentiment_name = sentimentLabels[sentiment_num]

    for category, sentiment_list in sentimentMap.items():
        if sentiment_name in sentiment_list:
            bucket = category
            break

    return {'sentiment': bucket}

goEmotionsDataset['train'] = goEmotionsDataset['train'].map(map_sentiment)

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Init VADER analyzer
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores("I love coding in Python!")

In [ ]:
# Update goEmotions dataset with VADER sentiment prediction

def get_vader_prediction(row):
    text = row['text']

    compound = analyzer.polarity_scores(text)['compound']

    if compound >= 0.05:
        prediction = "positive"
    elif compound <= -0.05:
        prediction = "negative"
    else:
        prediction = "neutral"

    return {"vader_prediction": prediction}

goEmotionsDataset['train'] = goEmotionsDataset['train'].map(get_vader_prediction)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true=goEmotionsDataset['train']['sentiment'], y_pred=goEmotionsDataset['train']['vader_prediction']))